In [125]:
# load API key
import os
from dotenv import load_dotenv
import openai

# to log
import logging

from typing import List

# parse json file with transcribed videos
import json

from llama_index import Document, VectorStoreIndex
from llama_index.llms import OpenAI
from llama_index.evaluation import DatasetGenerator, RelevancyEvaluator, FaithfulnessEvaluator

from llama_index import StorageContext, load_index_from_storage
from llama_index import ServiceContext

import pandas as pd
from llama_index.llms import OpenAI

from llama_index.node_parser import SimpleNodeParser


In [2]:
# make the async code working in the notebook cells
import nest_asyncio

nest_asyncio.apply()

In [3]:
# get the API key to the variable
load_dotenv(dotenv_path='../.env')

openai.api_key = os.getenv("OPENAI_API_KEY")

In [5]:
# set the logging
logging.basicConfig(filename='question_generation.log',  # Name of the log file
                    level=logging.DEBUG,     # Logging level
                    format='%(asctime)s - %(levelname)s - %(message)s',  # Format of log messages
                    filemode='w')  # Overwrite the log file on each script run

logging.debug('Run started')

some of the videos would have more than one node, and the last node may be too short to be used for the questions. We need to discard such nodes.

In [ ]:
def process_nodes(nodes):
    """
    Processes a list of nodes based on the following criteria:
    
    1. If a node has the same title as the previous one, it's added to the result list 
       only if its word count is more than 50% of the previous node's word count.
    2. If a node has a different title, it's simply added to the result list.

    Parameters:
    - nodes (list): A list of BaseNode objects.

    Returns:
    - list: A processed list of BaseNode objects.
    """
    
    # Initialize an empty list to store the processed nodes
    result = []

    # Initialize variables to keep track of the previous node's title and word count
    prev_title = None
    prev_word_count = 0

    # Iterate through the list of nodes
    for node in nodes:
        current_title = node.metadata['title']
        current_word_count = len(node.text.split())

        # Check if the current node has the same title as the previous one
        if current_title == prev_title:
            if current_word_count > 0.5 * prev_word_count:
                result.append(node)
        else:
            result.append(node)

        # Update the previous title and word count for the next iteration
        prev_title = current_title
        prev_word_count = current_word_count

    return result


In [111]:
# open json with transcribed videos and parse
with open(f"../data/video_info.json", "r", encoding="utf-8") as f:
    data = json.load(f)

data = data[:4]

docs = [
    Document(
        text="".join(d["text"]),
        metadata={"url": d["url"][0], "title": d["title"][0]},
    )
    for d in data
]

# set chunk_size and parse the docs into the nodes
chunk_size = 1024 * 8
parser = SimpleNodeParser.from_defaults(chunk_size=1024*8)

nodes = parser.get_nodes_from_documents(docs)

In [ ]:
# initialize the model
gpt3 = OpenAI(temperature=0, model="gpt-3.5-turbo")
service_context_gpt3 = ServiceContext.from_defaults(llm=gpt3, node_parser=parser)

Вариант 1 - сносно

In [138]:
question_gen_query = "Ты - начинающий спецалист в анализе данных. \
                        На основе приведенного текста составь только один вопрос, \
                        на который можно ответить с помощью текста. \
                        Вопрос должен покрыть как можно больше аспектов в тексте. \
                        Он должен быть только на основе привденного текста \
                        и относиться к области анализа данных" 

data_generator_nodes = DatasetGenerator(
    nodes=nodes[:2],
    service_context=service_context_gpt3,
    question_gen_query=question_gen_query
    )

eval_questions_nodes = data_generator_nodes.generate_questions_from_nodes()

eval_questions_nodes


['Какие задачи можно решить с помощью ML-симулятора?',
 'Какие данные хранятся в базе данных PostgreSQL?']

Вариант 2 - лучший

In [143]:
question_gen_query = "На основе приведенного текста составь только один вопрос, \
                        на который можно ответить с помощью текста. \
                        Вопрос должен покрыть как можно больше аспектов в тексте. \
                        Он должен быть только на основе привденного текста \
                        и относиться к области анализа данных" 

data_generator_nodes = DatasetGenerator(
    nodes=nodes[:2],
    service_context=service_context_gpt3,
    question_gen_query=question_gen_query
    )

eval_questions_nodes = data_generator_nodes.generate_questions_from_nodes()

eval_questions_nodes


['Какие задачи решает ML-симулятор и как он помогает инженеру по машинному обучению?',
 'Какие важные нюансы нужно учитывать при хранении данных в Redash и как это может повлиять на аналитический анализ данных?']

Вариант 3 - дает больше одного

In [140]:
question_gen_query = "Я хочу создать список эталонных вопросов к корпусу текстов. \
                      Эталонный вопрос относится к области анализа данных\
                      и на него можно ответить с помощью соответствующего текста.\
                      Вопрос должен покрыть как можно больше аспектов в тексте.\
                      Составь один эталонный вопрос к этому тексту."


data_generator_nodes = DatasetGenerator(
    nodes=nodes[:2],
    service_context=service_context_gpt3,
    question_gen_query=question_gen_query
    )

eval_questions_nodes = data_generator_nodes.generate_questions_from_nodes()

eval_questions_nodes


['Какие задачи решает ML-симулятор?',
 'Какие данные хранятся в специальной базе данных в PostgreSQL?',
 'Как Redash отображает результаты запросов?',
 'Как Redash отображает время в своем формате?',
 'Какие форматы отображения даты доступны в Redash?',
 'Какие нюансы связаны с отображением времени в Redash?',
 'Какие проблемы могут возникнуть при отборе записей по времени в Redash?',
 'Как данные хранятся в базе данных PostgreSQL?',
 'Какой формат времени используется в базе данных PostgreSQL?',
 'Какие форматы времени поддерживает Redash?',
 'Какие различия между форматом хранения данных и их отображением в Redash?']

Вариант 4 - сносно

In [141]:
question_gen_query = "На русском языке. только один вопрос на текст! Это должен быть именно один вопрос"

data_generator_nodes = DatasetGenerator(
    nodes=nodes[:2],
    service_context=service_context_gpt3,
    question_gen_query=question_gen_query
    )

eval_questions_nodes = data_generator_nodes.generate_questions_from_nodes()

eval_questions_nodes


['Какие задачи можно решить с помощью ML-симулятора?',
 'Какие данные хранятся в базе данных PostgreSQL?']

Вариант 5 - твой вариант - больше двух вопросов на ноду

In [142]:
data_generator_nodes = DatasetGenerator(
    nodes=nodes[:2],
    service_context=service_context_gpt3,
    num_questions_per_chunk=2,
    question_gen_query="На русском языке"
    )

eval_questions_nodes = data_generator_nodes.generate_questions_from_nodes()

eval_questions_nodes


['Что такое Симулятор ML и для чего он нужен?',
 'Какие задачи можно решить с помощью Симулятора ML?',
 'Какой опыт можно получить, проходя Симулятор ML?',
 'Как Симулятор ML поможет при приеме на работу?',
 'Как Симулятор ML поможет при выполнении новых задач на работе?',
 'Что такое Redash и как он отображает данные?',
 'Где хранятся данные в Redash?',
 'Как Redash отображает время?',
 'Какой формат отображения даты использует Redash?',
 'Какие нюансы есть в отображении данных в Redash?',
 'Какие настройки есть в Redash для отображения даты?',
 'Какие форматы даты можно использовать в Redash?',
 'Какие данные хранятся в базе данных в формате PostgreSQL?',
 'Какие проблемы могут возникнуть при отборе записей по времени в Redash?',
 'Какие инструменты существуют для визуализации данных, похожие на Redash?']

Вариант 6 - пример того, как на некоторые ноды бот не приводит вопрос (см вторую ноду)

In [137]:
data_generator_nodes_proc = DatasetGenerator(
    nodes=processed_nodes,
    service_context=service_context_gpt3,
    # num_questions_per_chunk=1,
    question_gen_query=question_gen_query
    )

eval_questions_nodes_proc = data_generator_nodes_proc.generate_questions_from_nodes()

eval_questions_nodes_proc

['Какие задачи можно решить с помощью ML-симулятора?',
 'Если бы мы просто перевели формат колонки Creation Time в текстовый формат, мы могли бы увидеть, как она выглядит. Однако, в Redash текстовый формат уже не форматирует данные, поэтому мы не можем узнать, как они хранятся в базе данных PostgreSQL.',
 'Как можно интерпретировать график и сделать статистический вывод?',
 'Какие задачи решает аналитик данных в банках?',
 'Какие факторы могут влиять на то, что товарный запас не был распродан в некоторых магазинах?',
 'Какие детали и нюансы вы заметили при рассмотрении задач других аналитиков?',
 'Какие проблемы с организацией работы и атмосферой были упомянуты девочкой, которая работала в магните?',
 'Какие проблемы возникли при выполнении проектов и как вы планируете к ним подходить в будущем?']